In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using UpROOT # reading ROOT files
using Parameters # pack, unpack
using TypedTables # 
using StaticArrays # fixed size arrays
using Plots # plotting
using LinearAlgebra # norm for a vector
using DelimitedFiles # read and write tabled data
using Test

In [ ]:
theme(:wong, bottom_margin=3Plots.PlotMeasures.mm) # colors of the plots

# Data exploration: $\eta\pi$ from COMPASS

In [ ]:
# download("https://apearce.web.cern.ch/apearce/aman-intro/etapi_predicts.root",
    # joinpath("data", "etapi_predicts.root"))

In [ ]:
f = TFile(joinpath("data","etapi_predicts.root")) # open the file
t = f["pi03pic"][:]; # get the tree from the disc to the memory

In [ ]:
propertynames(t) # list of branches, the names of columns

In [ ]:
histogram(t.pPimZ, bins=100, xlab="pz(π⁻) (GeV)")

Mda, what should I expect??

## 1. Apgebra of four vectors
Define the particle $(E,\vec p)$:
 - invariant mass function, and basic transformations

In [ ]:
@with_kw mutable struct Particle
    E::Float64
    p::MVector{3,Float64}
end
Particle(E,px,py,pz) = Particle(E=E, p=SVector(px,py,pz))
Particle(px,py,pz; msq=error("mass needed")) = Particle(E=sqrt(px^2+py^2+pz^2+msq), p=SVector(px,py,pz))

# properties
invmasssq(p::Particle) = p.E^2-sum(abs2,p.p)
mass(p::Particle) = sqrt(invmasssq(p))
# 
import Base:+,-,*,show
+(p1::Particle, p2::Particle) = Particle(E=p1.E+p2.E, p=p1.p+p2.p)
*(p::Particle, α::T where T<:Real) = Particle(E=p.E*α, p=p.p*α)
*(α::T where T<:Real, p::Particle) = Particle(E=p.E*α, p=p.p*α)
-(p1::Particle, p2::Particle) = p1+(-1)*p2
# 
show(io::IO, p::Particle) = print(io, "P($(p.E),$(p.p))")

# transformations
function Ry!(p,θ)
    sinθ, cosθ = sin(θ), cos(θ)
    p.p[1], p.p[3] =  p.p[1]*cosθ+sinθ*p.p[3],
                     -p.p[1]*sinθ+cosθ*p.p[3]
end
function Rz!(p,ϕ)
    sinϕ, cosϕ = sin(ϕ), cos(ϕ)
    p.p[1], p.p[2] = p.p[1]*cosϕ-sinϕ*p.p[2],
                     p.p[1]*sinϕ+cosϕ*p.p[2]
end
function Bz!(p,γ)
    _γ = abs(γ)
    _βγ = sqrt(γ^2-1)*sign(γ)
    p.p[3], p.E = _γ *p.p[3]+_βγ*p.E,
                  _βγ*p.p[3]+ _γ*p.E
end

Exercise: $\pi^0\to \gamma\gamma$ decay:
 - get an example of two four vectors of $p_{\gamma1}$ and $p_{\gamma2}$ that are decays of the
$$
\begin{align}
p_{\pi0} = (E,1,-1,30), \text{ with } E = \sqrt{30^2+1^2+1^2+m_{\pi}^2}
\end{align}
$$

In [ ]:
const mπ0 = 0.135
pπ0 = Particle(1,1,30; msq = mπ0^2)
pγ1 = Particle(10,0,0,10); pγ2 = Particle(20,0,0,10) # is incorrect choise

In [ ]:
@test invmasssq(pγ1) == 0.0

In [ ]:
@test invmasssq(pγ2) == 0.0

In [ ]:
@test invmasssq(pγ1+pγ2) == invmasssq(pπ0)

In [ ]:
@test sum((pγ1+pγ2).p .== pπ0.p) == 3

## Presentation of the data

#### Converting the TTree to a table of `Particles`
row by row

In [ ]:
process(row) = 
   (pb = Particle(row.pBeamE,   row.pBeamX,   row.pBeamY,   row.pBeamZ),
    pr = Particle(row.pRecoilE, row.pRecoilX, row.pRecoilY, row.pRecoilZ),
    pπ = Particle(row.pPimE,    row.pPimX,    row.pPimY,    row.pPimZ),
    pη = Particle(row.pEtaE,    row.pEtaX,    row.pEtaY,    row.pEtaZ))

In [ ]:
fv = Table(process.(t)); # create the four vectors

<img src="figs/etapi_d.PNG" alt="etapi" style="width: 300px;"/>

The scattering amplitude dependence on
$$
\begin{align}
s &= (p_\eta + p_\pi)^2\\
t_1 &= (p_\eta + p_{b})^2\\
t &= (p_{t} - p_r)^2\\
\end{align}
$$
The other two:
 - $s_0 = (p_b+p_t)^2$
 - $s_2 = (p_r+p_\pi)^2$
 Compute them all:

In [ ]:
plot(layout=grid(1,3), size=(900,250),
    histogram(mass.(fv.pπ+fv.pη+fv.pr), bins=100, xlab="√s₀ (GeV)"),
    histogram(mass.(fv.pπ+fv.pη), xlab="m(ηπ) (GeV)", bins=100),
    histogram(-invmasssq.(fv.pb-fv.pη-fv.pπ), xlab="transf. momentum, -t (GeV²)", bins=100, yscale=:log10)
)

In [ ]:
histogram2d(mass.(fv.pπ+fv.pη),
            invmasssq.(fv.pb-fv.pη), xlab="m(ηπ) (GeV)", ylab="t₁=(pb-pη)² (GeV²)",
    color=cgrad(:viridis, scale=:exp))

In [ ]:
angles(p::Particle) = p.p[3]/norm(p.p), atan(p.p[2],p.p[1])
boostzfactor(p::Particle) = p.E/mass(p)

## 3. Getting the right angles
Introducing the Gottfried-Jackson frame

<img src="figs/GJ.PNG" alt="GJ angles" style="width: 800px;"/>


Here:
$$
p_\eta = p_2+p_3,\quad p_1 = p_\pi
$$

Get the angle $\theta = \theta_1$ and $\phi = \phi_1$

Plotting recipe for the particle

In [ ]:
@recipe function f(p::Particle, proj=:zx)
    arrow   := true
    linewidth  :=  2
    tip = (p.p[3],p.p[1])
    tip = (proj == :xy) ? (p.p[1],p.p[2]) : tip
    return [(0,0), tip]
end
function plot_particles(pb, pη, pπ, pr)
    plot(layout=grid(1,2, widths=(0.7,0.3)), aspect_ratio=1, size=(850,280), frame=:origin)
    plot!(sp=1, xlab="z", ylab="x");    plot!(sp=2, xlab="x", ylab="y")
    plot!(sp=1, pb, :zx, lab="beam");   plot!(sp=2, pb, :xy, lab="")
    plot!(sp=1, pη, :zx, lab="η");      plot!(sp=2, pη, :xy, lab="")
    plot!(sp=1, pπ, :zx, lab="π");      plot!(sp=2, pπ, :xy, lab="")
    plot!(sp=1, pr, :zx, lab="recoil"); plot!(sp=2, pr, :xy, lab="")
end

In [ ]:
@unpack pb, pη, pπ, pr = fv[6]
plot_particles(pb, pη, pπ, pr)

In [ ]:
@unpack pb, pη, pπ, pr = fv[6]
cosθ,ϕ = angles(pπ+pη)
γ = boostzfactor(pπ+pη)
#
Rz!.([pb, pη, pπ, pr], -ϕ)
Ry!.([pb, pη, pπ, pr], -acos(cosθ))
Bz!.([pb, pη, pπ, pr], -γ)
# #
cosθ,ϕ = angles(pb)
Rz!.([pb, pη, pπ, pr], -ϕ)
Ry!.([pb, pη, pπ, pr], -acos(cosθ))
# # 
cosθ,ϕ = angles(pr)
Rz!.([pb, pη, pπ, pr], π-ϕ)
# 
plot_particles(pb, pη, pπ, pr)

In [ ]:
function anglesGJ!(;pb, pη, pπ, pr)
    pb, pη, pπ, pr
    cosθ,ϕ = angles(pπ+pη)
    γ = boostzfactor(pπ+pη)
    #
    Rz!.([pb, pη, pπ, pr], -ϕ)
    Ry!.([pb, pη, pπ, pr], -acos(cosθ))
    Bz!.([pb, pη, pπ, pr], -γ)
    # #
    cosθ,ϕ = angles(pb)
    Rz!.([pb, pη, pπ, pr], -ϕ)
    Ry!.([pb, pη, pπ, pr], -acos(cosθ))
    # # 
    cosθ,ϕ = angles(pr)
    Rz!.([pb, pη, pπ, pr], π-ϕ)
    return angles(pη)
end

In [ ]:
kinvars = Table([NamedTuple{(:cosθ,:ϕ,:mηπ)}([anglesGJ!(;v...)...,mass(v.pη+v.pπ)]) for v in fv])

In [ ]:
plot(layout=grid(1,3), size=(900,250),
    histogram(kinvars.cosθ, lab="", xlab="cosθ"),
    histogram(kinvars.ϕ, lab="", xlab="ϕ"),
    histogram(kinvars.mηπ, lab="", xlab="mηπ", bins=100)
)

## Explore how the angular dependence changes with mass

In [ ]:
histogram2d(kinvars.mηπ, kinvars.cosθ, xlab="m(ηπ) (GeV)", ylab="cosθ",
        color=cgrad(:viridis, scale=:exp))

In [ ]:
histogram2d(kinvars.cosθ, kinvars.ϕ, xlab="cosθ", ylab="ϕ", bins=100)

In [ ]:
writedlm(joinpath("data","data_metapi_costheta_phi.txt"), [kinvars.mηπ kinvars.cosθ kinvars.ϕ])